# Ejemplo: Creación de POJOs a partir de una consulta SPARQL y Ontología

**Fecha:** 21/04/2020

**Autor:** Guillermo Facundo Colunga, WESO Research Group, University of Oviedo. thewilly.work@gmail.com

# 1. Introducción

En este documento se explica como se pueden crear objetos POJO en java que representen la información extraida de una consurtal sparql y una ontología. Para este ejemplo y por ser más concretos se empleará el caso de ejemplo del proyecto Hércules. En este proyecto tenemos una ontología, que representa el esquema o valor semántico de los datos. Y unas preguntas de competencia que resultan la base de la funcionalidad del sistema. Por tanto para modelar los objetos que encapsularán los datos será necesario explorar las preguntas de competencia y la ontología.

# 2. Análisis de preguntas de competencia

Lo primero que hacemos es seleccionar una preguntad de competencia, en este caso seleccionaremos la [Q004](https://github.com/weso/hercules-ontology/issues/16), a continuación la descomponemos y formulamos una consulta sparql para ella. De esta forma somos capaces de ver qué información precisa la consulta para ser resuelta.

**Enunciado:** Como usuario requiero obtener el Top 10 (o el número que se considere relevante pues será parametrizable) de centros/estructuras de investigación que posean sellos de calidad asociados, por ejemplo: el sello Severo Ochoa.

**Elementos:** Del enunciado anterior somos capaces de abstraer que necesitamos consultar la lista de **Centros/Estructuras de Investigación** y filtrar por aquellos que posean **Sellos de Calidad Asociados** de algún tipo (nos indican que existe más de uno). Por lo tanto esta consulta requiere de dos elementos para poder resulverla. 1. Una entidad que incluya centro / Estructuras de Investigación. 2. Un elemento/propiedad que de alguna forma nos indique si la entidad descrita en (1) tiene sellos de calidad o no.

## 2.1 Consulta SPARQL

El siguiente paso y antes de poder generar la consulta sparql es ir a la ontología y buscar las clases/propiedades que se corresponden con los elementos que identificamos en el punto anterior. Lo primero que encontramos es que todos los centros o estructuras de investigación están amparadas bajo una misma clase que se llama [ResearchInstitute](https://github.com/weso/hercules-ontology/blob/master/src/asio-core.ttl#L1084-L1098), también nos encontramos que existe una clase que representa el [Sello de Calidad Asociado](https://github.com/weso/hercules-ontology/blob/master/src/asio-core.ttl#L1188-L1201) y por último encontramos una propiedad que define el facto de [tener un sello de calidad](https://github.com/weso/hercules-ontology/blob/master/src/asio-core.ttl#L185-L189)

Por lo tanto para nuestra query lo que buscamos son entidades de tipo ResearchInstitute que contengan propiedades de tipo hasSealOfQuality, recordamos que en la ontología trabajamos en Open World y por tanto podemos tener propiedades repetidas. Entonces, la consulta se podría expresar de la siguiente forma:


```sparql
PREFIX wb: <http://hercules-demo.wiki.opencura.com/entity/>
PREFIX wbt: <http://hercules-demo.wiki.opencura.com/prop/direct/>

SELECT ?centreLabel (COUNT(?seal) AS ?numberOfSeals) WHERE {
  ?centre wbt:P1 wb:Q2 ; # get entities of type ResearchInstitute
          wbt:P2 ?seal . # with property hasSealOfQuality
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "es" .
  }
}
GROUP BY ?centreLabel
ORDER BY DESC(?numberOfSeals)
LIMIT 10
```

Si analizamos esta consulta lo que vemos es que generamos 2 prefijos, simplemente para poder acceder a nuestra instancia privada de datos. A continuación accedemos a todas las entidades de tipo research-institute, que se corresponde en nuestro caso con la Q2 (esto es lo que gestionaría la factoría de URIs). Y de todas las entidades seleccionamos sólo aquellas que contienen 1 o N veces la propiedad P2 (gestionado por la factoría de URIs) que representa "tiene sello de calidad". Una vez aplicado el filtro agrupamos por el nombre de cada centro y mostramos en orden descendente (por # de sellos de calidad) los centros/estructuras de investigación.

# 3. Obtención de POJOs

Una vez hemos sido capaces de realizar nuestra consulta sparql, esto implica haber obtenido información semántica de la ontología, podemos modelar los objetos pojo que necesitamos para sustentar esta query. De forma que tenemos:

```java

public class ResearchInstitute() {
    
    private Map<String, String> label;            // The name of the research institute. It is a map due to locale. One entry per locale.
    private Map<String, String> description;      // The description of the research institute. It is a map due to locale. One entry per locale.
    private List<SealOfQuality> qualitySeals;     // The quality seals that the research institute owns.
    // More properties here.
    
    // Constructors...
    
    // Getters / Setters ...
}

public class QualitySeal() {
    
    private Map<String, String> label;            // The name of the research institute. It is a map due to locale. One entry per locale.
    private Map<String, String> description;      // The description of the research institute. It is a map due to locale. One entry per locale.
    // More properties here.
    
    // Constructors...
    
    // Getters / Setters ...
    
}

```

> **Nota:** Es importante darse cuenta de que estos POJOs representan el estado actual de la ontología, esto no implica que mañana se pueda modificar la ontología para incluis información, por ejemplo sobre cuando se concede un sello de calidad, generando una nueva clase que contenga la relación.

Y de esta forma si quiéramos conseguir las entidades que tienen al menos un selñlo de calidad haríamos:

```java
researchInstitutes.filter( getQualitySeals().size > 0)
```